# Taller inicial de Julia

### Environments and the Package manger (Pkg)

In [5]:
using Pkg
Pkg.activate(".")
Pkg.add("Parameters")
Pkg.add("Unitful")
Pkg.add("UnitfulAstro")
Pkg.add("Zygote")
Pkg.add("DataFrames")
Pkg.add("DifferentialEquations")

  Activating project at `/mnt/md1/mmestre/work/2023/TallerJulia`
   Resolving package versions...


  No Changes to `/mnt/md1/mmestre/work/2023/TallerJulia/Project.toml`
  No Changes to `/mnt/md1/mmestre/work/2023/TallerJulia/Manifest.toml`


Precompiling project...


  ✓ Distributions → DistributionsChainRulesCoreExt


  ✓ DiffEqNoiseProcess


...

  No Changes to `/mnt/md1/mmestre/work/2023/TallerJulia/Project.toml`
  No Changes to `/mnt/md1/mmestre/work/2023/TallerJulia/Manifest.toml`


   Resolving package versions...


  No Changes to `/mnt/md1/mmestre/work/2023/TallerJulia/Project.toml`
  No Changes to `/mnt/md1/mmestre/work/2023/TallerJulia/Manifest.toml`


   Resolving package versions...


  No Changes to `/mnt/md1/mmestre/work/2023/TallerJulia/Project.toml`
  No Changes to `/mnt/md1/mmestre/work/2023/TallerJulia/Manifest.toml`


In [49]:
# Arrays
a = [2.0, 3.0, 1.0]
@show a typeof(a)
@show typeof(a)<:Vector{Integer}
@show typeof(a)<:Vector{Float64}
suma = sum(a)
b = 7a
@show c = a + b
@show c = a + b.*b
@show c = a .+ b'b


a = [2.0, 3.0, 1.0]
typeof(a) = Vector{Float64}
typeof(a) <: Vector{Integer} = false
typeof(a) <: Vector{Float64} = true
c = a + b = [16.0, 24.0, 8.0]
c = a + b .* b = [198.0, 444.0, 50.0]
c = a .+ b' * b = [688.0, 689.0, 687.0]


3-element Vector{Float64}:
 688.0
 689.0
 687.0

In [34]:
m = [a 2a 3a]
display(m)
display(m')

3×3 Matrix{Float64}:
 2.0  4.0  6.0
 3.0  6.0  9.0
 1.0  2.0  3.0

3×3 adjoint(::Matrix{Float64}) with eltype Float64:
 2.0  3.0  1.0
 4.0  6.0  2.0
 6.0  9.0  3.0

In [30]:
display(m)

3×3 Matrix{Float64}:
 2.0  4.0  6.0
 3.0  6.0  9.0
 1.0  2.0  3.0

In [24]:
mp[2,2]

6.0